In [3]:
import pandas as pd
import datetime
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

%matplotlib inline

In [4]:
url = 'https://raw.githubusercontent.com/ngophuong/VNStock/main/excel_tcb.csv'
df = pd.read_csv(url, index_col = 0)
df['datetime']=pd.to_datetime(df['<DTYYYYMMDD>'],format='%Y%m%d')
df['Week_no']= df['datetime'].dt.week
df['Year_no']= df['datetime'].dt.year
df['Period'] = df['Year_no'].astype(str) + str("-") + df['Week_no'].astype(str)
df.head(3)

<ipython-input-4-7ef9f6d38ffb>:4: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,<Open>,<High>,<Low>,<Close>,<VolumeDeal>,<VolumeFB>,<VolumeFS>,datetime,Week_no,Year_no,Period
<Ticker>,,,,,,,,,,,,,,,,,
TCB,20210329,39.70,39.75,39.15,39.55,8380600,39.70,39.75,39.15,39.55,4145400,0,0,2021-03-29,13,2021,2021-13
TCB,20210326,39.15,39.65,38.40,39.40,10188400,39.15,39.65,38.40,39.40,1335000,0,0,2021-03-26,12,2021,2021-12
TCB,20210325,39.40,39.80,39.20,39.35,10420800,39.40,39.80,39.20,39.35,20000,0,0,2021-03-25,12,2021,2021-12


In [5]:
df =df[['<Close>','datetime','<Volume>','Period','Year_no']]
df = df.head(1500)

In [6]:
#plot linechart for stock price
short_rolling = df['<Close>'].rolling(window=20).mean()

ema_long = df['<Close>'].ewm(span=50, adjust=False).mean()
ema_short = df['<Close>'].ewm(span=20, adjust=False).mean()
line_price = df['<Close>']
fig = go.Figure()
#px.line(df_hpg, x='datetime', y='<Close>', labels={'x':'Date', 'y':'Price'})

fig.add_trace(go.Scatter(x=df.datetime, y=line_price, 
                        mode='lines', name='TCB'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_short, 
                        mode='lines', name='Span 20-days EMA'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_long, 
                        mode='lines', name='Span 50-days EMA'))


In [7]:
#px.violin(df_tips, y="tip", x="smoker", color="sex", box=True, points="all",
#          hover_data=df_tips.columns)

fig = go.Figure()
px.violin(df,x='Year_no',y='<Close>',box =True, points='all',
         hover_data=df.columns, title = 'VND Violin chart')

In [8]:
df.tail(3)

,<Close>,datetime,<Volume>,Period,Year_no
<Ticker>,,,,,
TCB,92.0,2018-06-06,1605480,2018-23,2018
TCB,96.0,2018-06-05,1357500,2018-23,2018
TCB,102.4,2018-06-04,2811840,2018-23,2018


In [9]:
#HDFC_df[‘Day_Perc_Change’] = HDFC_df[‘Adj Close’].pct_change()*100
df['Daily_return']=df['<Close>'].pct_change()*100
df.dropna(axis = 0, inplace = True)
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return
<Ticker>,,,,,,
TCB,39.40,2021-03-26,10188400,2021-12,2021,-0.379267
TCB,39.35,2021-03-25,10420800,2021-12,2021,-0.126904
TCB,39.30,2021-03-24,13792100,2021-12,2021,-0.127065


In [10]:
fig = go.Figure()
px.line(df, x='datetime', y='Daily_return', labels={'x':'Date', 'y':'Daily_Return_%'})

In [11]:
df['Daily_return'].describe()

count    706.000000
mean       0.272639
std        7.367829
min       -6.504065
25%       -0.960962
50%        0.000000
75%        0.859570
max      187.744227
Name: Daily_return, dtype: float64

In [12]:
fig = go.Figure()
px.histogram(df, x="Daily_return",histnorm='percent')
#ff.create_distplot(df, group_labels)

In [13]:
df['<Close>'].describe()

count    706.000000
mean      26.886898
std       13.742693
min       14.900000
25%       21.362500
50%       23.650000
75%       26.800000
max      109.000000
Name: <Close>, dtype: float64

In [14]:
dfig = go.Figure()
px.histogram(df.head(60), x="Daily_return",histnorm='percent')

In [15]:


df['Volumn'] = df['<Volume>']/1000000
Volumnshort_rolling = df['Volumn'].rolling(window=7).mean()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datetime, y=df['Daily_return'], 
                        mode='lines', name='Dailly_Retun'))
fig.add_trace(go.Scatter(x=df.datetime, y=df['Volumn'], 
                        mode='lines+markers', name='Volumn'))

fig.add_trace(go.Scatter(x=df.datetime, y=Volumnshort_rolling, 
                        mode='lines', name='Rolling_volumn'))

In [16]:
df['Volumn'].head(15).describe()

count    15.000000
mean     11.026880
std       2.604244
min       7.413700
25%       9.296400
50%      10.554500
75%      12.287500
max      16.410100
Name: Volumn, dtype: float64

In [17]:
df['Volumn'].head(10).describe()

count    10.000000
mean     11.153590
std       2.643808
min       7.413700
25%       9.704200
50%      10.487650
75%      12.642600
max      16.410100
Name: Volumn, dtype: float64